In [1]:
#%%
import os
import numpy as np 
import pandas as pd 
import scanpy as sc 
import matplotlib.pyplot as plt
from tqdm import tqdm  # Import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from torchmetrics import Accuracy

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
#%%
# Select platform
platform = "xenium" # xenium or visium 
ground_truth = "refined"  # refined or cellvit
level = 0
filtered_genes = False
label_source = "singleR"  # singleR, celltypist, aistil, combined
use_qc = False
morph_version = "v2"
use_projections = True
limit_classes = True  # Set to False to use all classes


if platform == "xenium":
    cancer = "lung"
    xenium_folder_dict = {"lung": "Xenium_Prime_Human_Lung_Cancer_FFPE_outs",
                          "breast":"Xenium_Prime_Breast_Cancer_FFPE_outs",
                          "lymph_node": "Xenium_Prime_Human_Lymph_Node_Reactive_FFPE_outs",
                          "prostate": "Xenium_Prime_Human_Prostate_FFPE_outs",
                          "skin": "Xenium_Prime_Human_Skin_FFPE_outs",
                          "ovarian": "Xenium_Prime_Ovarian_Cancer_FFPE_outs",
                          "cervical": "Xenium_Prime_Cervical_Cancer_FFPE_outs"
                          }

    xenium_folder = xenium_folder_dict[cancer]
    celltypist_data_path = f"/rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/public_data/10x_genomics/{xenium_folder}/preprocessed/fine_tune_{ground_truth}_v2/processed_xenium_data_fine_tune_{ground_truth}_ImmuneHigh_v2.h5ad"
    singleR_data_path = f"/rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/public_data/10x_genomics/{xenium_folder}/preprocessed/fine_tune_{ground_truth}_v2/processed_xenium_data_fine_tune_{ground_truth}_v2_annotated.h5ad"
    
    embedding_dir = f"/rsrch9/home/plm/idso_fa1_pathology/TIER2/paul-xenium/embeddings/public_data/{xenium_folder}"
    gene_emb_path = f"{embedding_dir}/scGPT/scGPT_CP.h5ad"

    if filtered_genes:
        gene_embedding_file = f"/rsrch9/home/plm/idso_fa1_pathology/TIER2/paul-xenium/embeddings/public_data/{xenium_folder}/processed_xenium_refined_clustering_filtered_v2.csv"
    else:
        gene_embedding_file = f"/rsrch9/home/plm/idso_fa1_pathology/TIER2/paul-xenium/embeddings/public_data/{xenium_folder}/processed_xenium_{ground_truth}_v2.csv"

    results_root = "/rsrch9/home/plm/idso_fa1_pathology/TIER2/paul-xenium/classification_results/public_data"
    results_dir = f"{results_root}/{xenium_folder}"
    os.makedirs(results_dir, exist_ok=True)
        
elif platform == "visium":
    data_path = "/rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/public_data/10x_genomics/Visium_HD_Human_Lung_Cancer_post_Xenium_Prime_5k_Experiment2/binned_outputs/square_002um/preprocessed/bin2cell/to_tokenize/corrected_cells_matched_preprocessed_refined_v2.h5ad"

    gene_embedding_file = "/rsrch9/home/plm/idso_fa1_pathology/TIER2/paul-xenium/embeddings/public_data/Visium_HD_Human_Lung_Cancer_post_Xenium_Prime_5k_Experiment2/bin2cell/embeddings_output/processed_visium_hd_bin2cell.csv"
    morph_embedding_dir = f"/rsrch9/home/plm/idso_fa1_pathology/TIER2/paul-xenium/embeddings/public_data/Visium_HD_Human_Lung_Cancer_post_Xenium_Prime_5k_Experiment2"



# Load AnnData
# if label_source == "singleR":
data_path = singleR_data_path
adata = sc.read_h5ad(data_path)

if label_source == "celltypist":
    data_path = celltypist
    adata = sc.read_h5ad(data_path)
    
elif label_source == "combined":
    adata = sc.read_h5ad(singleR_data_path)
    bdata = sc.read_h5ad(celltypist_data_path)
    adata.obs["majority_voting"] = bdata.obs["majority_voting"]
    adata.obs["qc_celltypist"] = bdata.obs["qc_celltypist"]
    cell_data = adata.obs

cell_data = adata.obs

# Spatial Information 
spatial_coords = cell_data[['x_centroid', 'y_centroid']].rename(columns={'x_centroid': 'x', 'y_centroid': 'y'})

# Load gene Embeddings 
if not use_projections:
    print("Using zero-shot embeddings")
    gdata = sc.read_h5ad(gene_emb_path)
    gene_embeddings = pd.DataFrame(gdata.obsm["X_scGPT"])
    gene_embeddings.index = cell_data.index
    
    # Load Morphology Embeddings 
    # Load Morphology Embeddings 
    if morph_version == "v1":
        morph_embedding_csv = os.path.join(embedding_dir, "UNI2_cell_representation",f"level_{level}","morphology_embeddings_v2.csv")  # morphology_embeddings_v2
    else:
        morph_embedding_csv = os.path.join(embedding_dir, "UNI2_cell_representation",f"level_{level}","uni2_pretrained_embeddings.csv")  # morphology_embeddings_v2
        
    morph_embeddings = pd.read_csv(morph_embedding_csv, index_col="Unnamed: 0")
else:
    print("Using contrastive projections")
    gene_embeddings = pd.read_csv(os.path.join(embedding_dir, "contrastive_learning", "gene_projection_embeddings_v2.csv"), index_col=0)
    morph_embeddings = pd.read_csv(os.path.join(embedding_dir, "contrastive_learning", "morph_projection_embeddings_v2.csv"), index_col=0)

if label_source=="singleR":
    print("Using labels from SingleR.")
    singleR_to_class_map = {
        "Smooth muscle": "fibroblast",
        "Fibroblasts": "fibroblast",
        "Endothelial cells": "endothelial",
        "CD4+ T-cells": "t_cell",
        "CD8+ T-cells": "t_cell",
        "B-cells": "b_cell",
        "Macrophages": "macrophage",
        "Epithelial cells": "epithelial",
    }
    
    target_classes = ["fibroblast", "endothelial",
                      "t_cell", "b_cell", "macrophage",
                      "epithelial"]
    
    # Map SingleR labels to 7-class system
    cell_data[label_source] = cell_data["singleR_class"].map(singleR_to_class_map)
    
    # Drop cells that are nan (if any)
    cell_data = cell_data.dropna(subset=[label_source])
    
    # Keep only those 7 classes
    cell_data = cell_data[cell_data[label_source].isin(target_classes)]
    
    if use_qc:
        cell_data = cell_data[cell_data["qc_singleR"]==1]

    
    # Reindex embeddings/coords
    gene_embeddings = gene_embeddings.reindex(cell_data.index)
    morph_embeddings = morph_embeddings.reindex(cell_data.index)
    spatial_coords = spatial_coords.reindex(cell_data.index)
    
    
elif label_source=="aistil":
    print("Using AISTIL labels")
    label_key = "class"
    target_classes = ["f", "l", "t"]  # Modify this list to restrict classification to specific classes
    if limit_classes:
        num_classes = len(target_classes)
        cell_data = cell_data[cell_data[label_key].isin(target_classes)]

        # Change index type for Visium data to match embeddings Idxs 
        if platform == "visium":
            morph_embeddings.index = morph_embeddings.index.astype(str)

        # Update corresponding embeddings and spatial coordinates
        gene_embeddings = gene_embeddings.reindex(cell_data.index)
        morph_embeddings = morph_embeddings.reindex(cell_data.index)
        spatial_coords = spatial_coords.reindex(cell_data.index)
    else:
        target_classes = ["f","l","o","t"]
        
elif label_source=="celltypist":
    print("Using CellTypist Labels")
    celltypist_to_class_map = {
        "Fibroblasts": "fibroblast",
        "Endothelial cells": "endothelial",
        "T cells": "t_cell",
        "B cells": "b_cell",
        "Macrophages": "macrophage",
        "Epithelial cells": "epithelial",
    }
    target_classes = ["fibroblast", "endothelial",
                      "t_cell", "b_cell", "macrophage",
                      "epithelial"]

    # Map SingleR labels to 7-class system
    cell_data[label_source] = cell_data["majority_voting"].map(celltypist_to_class_map)
    
    # Drop cells that are nan (if any)
    cell_data = cell_data.dropna(subset=[])
    
    # Keep only those 7 classes
    cell_data = cell_data[cell_data[label_source].isin(target_classes)]

    if use_qc:
        cell_data = cell_data[cell_data["qc_celltypist"]==1]

    # Reindex embeddings/coords
    gene_embeddings = gene_embeddings.reindex(cell_data.index)
    morph_embeddings = morph_embeddings.reindex(cell_data.index)
    spatial_coords = spatial_coords.reindex(cell_data.index)
    
elif label_source == "combined":
    print("Using combined SingleR and CellTypist labels (agreement only)")

    # Define the shared label map and target classes
    shared_class_map = {
        "Fibroblasts": "fibroblast",
        "Smooth muscle": "fibroblast",
        "Endothelial cells": "endothelial",
        "CD4+ T-cells": "t_cell",
        "CD8+ T-cells": "t_cell",
        "T cells": "t_cell",
        "B cells": "b_cell",
        "B-cells": "b_cell",
        "Macrophages": "macrophage",
        "Epithelial cells": "epithelial",
    }
    
    target_classes = ["fibroblast", "endothelial", "t_cell", "b_cell", "macrophage", "epithelial"]

    # First map the labels (these are safe)
    cell_data["singleR_mapped"] = cell_data["singleR_class"].map(shared_class_map)
    cell_data["celltypist_mapped"] = cell_data["majority_voting"].map(shared_class_map)
    
    # Then immediately filter with a properly aligned mask
    cell_data = cell_data[
        cell_data["singleR_mapped"].notnull() &
        cell_data["celltypist_mapped"].notnull() &
        (cell_data["singleR_mapped"] == cell_data["celltypist_mapped"])
    ].copy()
    

    # Rename the final label column
    cell_data["combined"] = cell_data["singleR_mapped"]
    
    if use_qc:
        qc_mask = cell_data["qc_singleR"] == 1
        if "qc_celltypist" in cell_data.columns:
            qc_mask &= cell_data["qc_celltypist"] == 1
        cell_data = cell_data[qc_mask]

    # Reindex everything to the filtered cells
    gene_embeddings = gene_embeddings.reindex(cell_data.index)
    morph_embeddings = morph_embeddings.reindex(cell_data.index)
    spatial_coords = spatial_coords.reindex(cell_data.index)



num_classes = len(target_classes)
label_mapping = {cls_name: i for i, cls_name in enumerate(target_classes)}
if label_source == "aistil":
    labels = pd.Series(cell_data[label_key].map(label_mapping))
else:
    labels = pd.Series(cell_data[label_source].map(label_mapping))

Using contrastive projections
Using labels from SingleR.


In [3]:
print("Cell data index:", cell_data.index.tolist()[:5])
print("Morph embeddings index:", morph_embeddings.index.tolist()[:5])
print("Gene embeddings index:", gene_embeddings.index.tolist()[:5])
print("Spatial coords index:", spatial_coords.index.tolist()[:5])
cell_data



Cell data index: ['aaaaadnb-1', 'aaaabalp-1', 'aaaadjia-1', 'aaaafglb-1', 'aaaagbdd-1']
Morph embeddings index: ['aaaaadnb-1', 'aaaabalp-1', 'aaaadjia-1', 'aaaafglb-1', 'aaaagbdd-1']
Gene embeddings index: ['aaaaadnb-1', 'aaaabalp-1', 'aaaadjia-1', 'aaaafglb-1', 'aaaagbdd-1']
Spatial coords index: ['aaaaadnb-1', 'aaaabalp-1', 'aaaadjia-1', 'aaaafglb-1', 'aaaagbdd-1']


,x_centroid_um,y_centroid_um,transcript_counts,control_probe_counts,genomic_control_counts,control_codeword_counts,unassigned_codeword_counts,deprecated_codeword_counts,total_counts,cell_area,...,log1p_total_counts_mt,pct_counts_mt,n_genes,x_scaled,y_scaled,n_counts,split,singleR_class,qc_singleR,singleR
aaaaadnb-1,822.469055,5111.537598,226,0,0,0,0,0,226.0,44.659533,...,0.0,0.0,187,0.071725,0.699141,226.0,train,CD4+ T-cells,1,t_cell
aaaabalp-1,843.901428,5149.261230,108,0,0,0,0,0,108.0,32.196407,...,0.0,0.0,101,0.073594,0.704301,108.0,train,CD8+ T-cells,1,t_cell
aaaadjia-1,839.742981,5159.693848,328,0,0,0,0,1,328.0,37.253908,...,0.0,0.0,256,0.073231,0.705728,328.0,eval,CD4+ T-cells,1,t_cell
aaaafglb-1,784.250916,5143.785645,337,0,0,0,0,0,337.0,52.155471,...,0.0,0.0,261,0.068392,0.703552,337.0,test,CD8+ T-cells,1,t_cell
aaaagbdd-1,793.676331,5164.925293,294,0,0,0,0,1,294.0,51.387814,...,0.0,0.0,244,0.069214,0.706443,294.0,train,CD4+ T-cells,1,t_cell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
oikejpmc-1,8514.373047,993.629089,40,0,0,0,0,1,40.0,16.978751,...,0.0,0.0,39,0.742508,0.135906,40.0,test,Macrophages,0,macrophage
oikeooje-1,8598.839844,748.067261,294,0,0,0,0,0,294.0,180.534694,...,0.0,0.0,235,0.749874,0.102318,294.0,eval,Fibroblasts,0,fibroblast
oikgaiei-1,8181.042480,1044.580200,16,0,0,0,0,0,16.0,66.153909,...,0.0,0.0,16,0.713439,0.142875,16.0,train,Macrophages,0,macrophage
oikhmgdh-1,8025.134766,1012.913635,12,0,0,0,0,0,12.0,21.042813,...,0.0,0.0,12,0.699843,0.138543,12.0,train,CD4+ T-cells,0,t_cell


In [ ]:

# Positional Encoding
class PositionalEncoding2D(nn.Module):
    """Sinusoidal positional encoding for spatial coordinates"""
    def __init__(self, d_model):
        super().__init__()
        self.d_model = d_model
        self.proj = nn.Linear(4, d_model)  # (sin(x), cos(x), sin(y), cos(y))
        
    def forward(self, coords):
        # Ensure coords is [B, 2]
        assert coords.ndim == 2, f"Expected coords shape [B, 2], but got {coords.shape}"
        x = coords[:, 0] * 2 * torch.pi
        y = coords[:, 1] * 2 * torch.pi
        
        pe = torch.stack([
            torch.sin(x), torch.cos(x),
            torch.sin(y), torch.cos(y)
        ], dim=-1)  # [B, 4]
        pe = self.proj(pe)  # [B, d_model]
        return pe

# Transformer Layer with Relative Position Attention
class RelativePositionTransformerLayer(nn.TransformerEncoderLayer):
    """Enhanced with relative position attention"""
    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1):
        # Initialize parent class with batch_first=True.
        super().__init__(d_model, nhead, dim_feedforward, dropout, batch_first=True)
        self.pos_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True)
        self.pos_norm = nn.LayerNorm(d_model)
        # Add a dropout for the feedforward branch
        self.dropout3 = nn.Dropout(dropout)

    def forward(self, src, pos_emb):
        # Uncomment to debug
        # print(f"src shape before attn: {src.shape}")   # Expected [B, 2, d_model]
        # print(f"pos_emb shape before attn: {pos_emb.shape}")  # Expected [B, 1, d_model]

        # Ensure pos_emb has the same sequence length as src.
        pos_emb = pos_emb.expand(-1, src.shape[1], -1)  # Now [B, 2, d_model]
        
        # Uncomment to debug
        # print(f"pos_emb shape after expansion: {pos_emb.shape}")  # Should be [B, 2, d_model]

        # Standard self-attention
        src2 = self.self_attn(src, src, src, need_weights=False)[0]
        src = src + self.dropout1(src2)
        src = self.norm1(src)

        # Position-aware attention
        src2 = self.pos_attn(src, pos_emb, pos_emb)[0]
        src = src + self.dropout2(src2)
        src = self.pos_norm(src)

        # Feedforward
        src2 = self.linear2(self.dropout(self.activation(self.linear1(src))))
        src = src + self.dropout3(src2)
        src = self.norm2(src)
        return src

# Cell Transformer
class CellTransformer(nn.Module):
    def __init__(self, d_model=512, num_heads=8, num_classes=3):
        super().__init__()
        # Modality projections
        gene_input_dim = 128 if use_projections else 512
        self.gene_proj = nn.Linear(gene_input_dim, d_model)

        morph_input_dim = 128 if use_projections else 1536
        self.morph_proj = nn.Linear(morph_input_dim, d_model)
        self.spatial_pe = PositionalEncoding2D(d_model)
        # Modality type embeddings
        self.gene_type = nn.Parameter(torch.randn(1, d_model))
        self.morph_type = nn.Parameter(torch.randn(1, d_model))
        # Transformer layers
        self.layers = nn.ModuleList([
            RelativePositionTransformerLayer(d_model, num_heads)
            for _ in range(6)
        ])
        # Classifier
        self.classifier = nn.Sequential(
            nn.LayerNorm(d_model),
            nn.Linear(d_model, num_classes)
        )

    def forward(self, gene, morph, spatial):
        # Uncomment to debug
        # print(f"Input gene shape: {gene.shape}")      # Expected: [B, 512]
        # print(f"Input morph shape: {morph.shape}")      # Expected: [B, 1536]
        # print(f"Input spatial shape: {spatial.shape}")  # Expected: [B, 2]

        # Project each modality and add type embedding
        gene_emb = self.gene_proj(gene) + self.gene_type   # [B, d_model]
        morph_emb = self.morph_proj(morph) + self.morph_type  # [B, d_model]
        spatial_emb = self.spatial_pe(spatial)              # [B, d_model]
        
        # Uncomment to debug
        # print(f"Gene shape after projection: {gene_emb.shape}")    # Should be [B, d_model]
        # print(f"Morph shape after projection: {morph_emb.shape}")    # Should be [B, d_model]
        # print(f"Spatial shape after encoding: {spatial_emb.shape}")  # Should be [B, d_model]

        # Stack gene and morph embeddings into tokens
        tokens = torch.stack([gene_emb, morph_emb], dim=1)  # [B, 2, d_model]
        # Unsqueeze spatial embedding for attention (to be expanded in the layer)
        spatial_emb = spatial_emb.unsqueeze(1)  # [B, 1, d_model]

        # Pass tokens through transformer layers
        for layer in self.layers:
            tokens = layer(tokens, spatial_emb)  # Expected: [B, 2, d_model]

        # Pool over tokens and classify
        pooled = tokens.mean(dim=1)  # [B, d_model]
        return self.classifier(pooled)

# Data Handling
class CellDataset(Dataset):
    def __init__(self, gene, morph, spatial, labels):
        self.gene = torch.tensor(gene.values, dtype=torch.float32)      # [N, 512]
        self.morph = torch.tensor(morph.values, dtype=torch.float32)      # [N, 1536]
        self.spatial = torch.tensor(spatial.values, dtype=torch.float32)  # [N, 2]
        self.labels = torch.tensor(labels.values, dtype=torch.long)
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        gene_sample = self.gene[idx]     # [512]
        morph_sample = self.morph[idx]     # [1536]
        spatial_sample = self.spatial[idx] # [2]
        label = self.labels[idx]
        return gene_sample, morph_sample, spatial_sample, label

def compute_class_weights(labels):
    counts = np.bincount(labels.values.astype(int))
    weights = 1. / (counts + 1e-8)  # Prevent division by zero
    return torch.tensor(weights, dtype=torch.float32)



# Training and testing loop

num_epochs = 20 
batch_size = 64

# Initialize device and model
device = torch.device("cuda")
model = CellTransformer(d_model=512, num_heads=8, num_classes=num_classes).to(device)

# Optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5, weight_decay=0.01)
dataset = CellDataset(gene_embeddings, morph_embeddings, spatial_coords, labels)

# Split dataset into 80% train and 20% test (stratified by labels)
all_indices = np.arange(len(dataset))
train_idx, test_idx = train_test_split(
    all_indices, test_size=0.2, random_state=42, stratify=dataset.labels.numpy()
)
train_dataset = Subset(dataset, train_idx)
test_dataset = Subset(dataset, test_idx)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=3e-5, steps_per_epoch=len(train_loader), epochs=num_epochs
)
# Loss function with class weights moved to the proper device
criterion = nn.CrossEntropyLoss(weight=compute_class_weights(labels).to(device))

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    correct = 0
    total = 0

    # Wrap the train_loader with tqdm for a progress bar
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=False)
    for batch_idx, (gene, morph, spatial, lbls) in enumerate(progress_bar):
        gene, morph, spatial, lbls = (
            gene.to(device),
            morph.to(device),
            spatial.to(device),
            lbls.to(device)
        )

        optimizer.zero_grad()
        outputs = model(gene, morph, spatial)
        loss = criterion(outputs, lbls)
        loss.backward()
        optimizer.step()
        scheduler.step()

        epoch_loss += loss.item() * gene.size(0)
        _, predicted = torch.max(outputs, 1)
        total += lbls.size(0)
        correct += (predicted == lbls).sum().item()

        # Update progress bar with current loss
        progress_bar.set_postfix(loss=loss.item())

    avg_loss = epoch_loss / total
    accuracy = correct / total * 100 
    print(f"Epoch {epoch+1}: Loss: {avg_loss:.4f} | Accuracy: {accuracy:.2f}%")

    # Evaluation on the test set
    model.eval()
    test_loss = 0.0
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for gene, morph, spatial, lbls in test_loader:
            gene, morph, spatial, lbls = (
                gene.to(device),
                morph.to(device),
                spatial.to(device),
                lbls.to(device)
            )
            outputs = model(gene, morph, spatial)
            loss = criterion(outputs, lbls)
            test_loss += loss.item() * gene.size(0)
            _, predicted = torch.max(outputs, 1)
            test_total += lbls.size(0)
            test_correct += (predicted == lbls).sum().item()
    avg_test_loss = test_loss / test_total
    test_accuracy = test_correct / test_total * 100
    print(f"Test: Loss: {avg_test_loss:.4f} | Accuracy: {test_accuracy:.2f}%")



Epoch 1:  40%|███▉      | 912/2294 [00:32<00:53, 25.92it/s, loss=1.84]

In [ ]:
# Eval on same data 
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for gene, morph, spatial, lbls in test_loader:
        gene = gene.to(device)
        morph = morph.to(device)
        spatial = spatial.to(device)
        lbls = lbls.to(device)

        outputs = model(gene, morph, spatial)
        preds = torch.argmax(outputs, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(lbls.cpu().numpy())

cm = confusion_matrix(all_labels, all_preds, normalize="true")

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=target_classes)


disp.plot(cmap='viridis', xticks_rotation='vertical')
plt.title(f"Confusion Matrix: {platform}")
plt.show()
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=target_classes))

In [ ]:
report_dict = classification_report(all_labels, all_preds, target_names=target_classes, output_dict=True)
report_df = pd.DataFrame(report_dict).transpose()

save_path = os.path.join(results_dir, "T4")
os.makedirs(save_path, exist_ok=True)
if not use_projections:
    report_df.to_csv(os.path.join(save_path, f"classification_report_{label_source}_{morph_version}.csv"))
else:
    report_df.to_csv(os.path.join(save_path, f"classification_report_{label_source}_{morph_version}_proj.csv"))

In [ ]:
# #%%Eval trained model on different data 
# # Select platform
# platform = "xenium" # xenium or visium 
# ground_truth = "refined"  # refined or cellvit
# level = 0

# #%% Define Class Limiting Parameters
# limit_classes = True  # Set to False to use all classes
# target_classes = ["f", "l", "t"]  # Modify this list to restrict classification to specific classes

# if platform == "xenium":
#     cancer = "breast"
#     xenium_folder_dict = {"lung": "Xenium_Prime_Human_Lung_Cancer_FFPE_outs",
#                           "breast":"Xenium_Prime_Breast_Cancer_FFPE_outs",
#                           "lymph_node": "Xenium_Prime_Human_Lymph_Node_Reactive_FFPE_outs",
#                           "prostate": "Xenium_Prime_Human_Prostate_FFPE_outs",
#                           "skin": "Xenium_Prime_Human_Skin_FFPE_outs",
#                           "ovarian": "Xenium_Prime_Ovarian_Cancer_FFPE_outs",
#                           "cervical": "Xenium_Prime_Cervical_Cancer_FFPE_outs"
#                           }

#     xenium_folder = xenium_folder_dict[cancer]
    
#     data_path = f"/rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/public_data/10x_genomics/{xenium_folder}/preprocessed/fine_tune_{ground_truth}_v2/processed_xenium_data_fine_tune_{ground_truth}_v2.h5ad"
#     gene_embedding_file = f"/rsrch9/home/plm/idso_fa1_pathology/TIER2/paul-xenium/embeddings/public_data/{xenium_folder}/processed_xenium_{ground_truth}_v2.csv"
    
#     # Load Morphological Embeddings
#     morph_embedding_dir = f"/rsrch9/home/plm/idso_fa1_pathology/TIER2/paul-xenium/embeddings/public_data/{xenium_folder}"

        
# elif platform == "visium":
#     data_path = "/rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/public_data/10x_genomics/Visium_HD_Human_Lung_Cancer_post_Xenium_Prime_5k_Experiment2/binned_outputs/square_002um/preprocessed/bin2cell/to_tokenize/corrected_cells_matched_preprocessed_refined_v2.h5ad"

#     gene_embedding_file = "/rsrch9/home/plm/idso_fa1_pathology/TIER2/paul-xenium/embeddings/public_data/Visium_HD_Human_Lung_Cancer_post_Xenium_Prime_5k_Experiment2/bin2cell/embeddings_output/processed_visium_hd_bin2cell.csv"
#     morph_embedding_dir = f"/rsrch9/home/plm/idso_fa1_pathology/TIER2/paul-xenium/embeddings/public_data/Visium_HD_Human_Lung_Cancer_post_Xenium_Prime_5k_Experiment2"

# # Load AnnData
# adata = sc.read_h5ad(data_path)
# cell_data = adata.obs

# # Spatial Information 
# spatial_coords = cell_data[['x_centroid', 'y_centroid']].rename(columns={'x_centroid': 'x', 'y_centroid': 'y'})

# # Load gene Embeddings 
# gene_embeddings = pd.read_csv(gene_embedding_file, index_col="Unnamed: 0")
# gene_embeddings.index = cell_data.index

# # Load Morphology Embeddings 
# morph_embedding_csv = os.path.join(morph_embedding_dir, "UNI2_cell_representation",f"level_{level}","morphology_embeddings_v2.csv")
# morph_embeddings = pd.read_csv(morph_embedding_csv, index_col="Unnamed: 0")


# if limit_classes:
#     num_classes = len(target_classes)
#     cell_data = cell_data[cell_data["class"].isin(target_classes)]
    
#     # Change index type for Visium data to match embeddings Idxs 
#     if platform == "visium":
#         morph_embeddings.index = morph_embeddings.index.astype(str)

#     # Update corresponding embeddings and spatial coordinates
#     gene_embeddings = gene_embeddings.reindex(cell_data.index)
#     morph_embeddings = morph_embeddings.reindex(cell_data.index)
#     spatial_coords = spatial_coords.reindex(cell_data.index)
# else:
#     target_classes = ["f","l","o","t"]
    
# # Convert labels to numerical indices
# label_mapping = {l:c for c,l in enumerate(target_classes)}
# labels = pd.Series([label_mapping[lbl] for lbl in cell_data["class"]])

# dataset = CellDataset(gene_embeddings, morph_embeddings, spatial_coords, labels)
# test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
# model.eval()
# all_preds = []
# all_labels = []

# with torch.no_grad():
#     for gene, morph, spatial, lbls in test_loader:
#         gene = gene.to(device)
#         morph = morph.to(device)
#         spatial = spatial.to(device)
#         lbls = lbls.to(device)

#         outputs = model(gene, morph, spatial)
#         preds = torch.argmax(outputs, dim=1)

#         all_preds.extend(preds.cpu().numpy())
#         all_labels.extend(lbls.cpu().numpy())

# cm = confusion_matrix(all_labels, all_preds)

# disp = ConfusionMatrixDisplay(confusion_matrix=cm,
#                               display_labels=target_classes)


# disp.plot(cmap='viridis', xticks_rotation='vertical')
# plt.title(f"Confusion Matrix: {platform}")
# plt.show()
# print("Classification Report:")
# print(classification_report(all_labels, all_preds))